In [1]:
import re
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import json

BUFFER_SIZE = 1500

In [2]:
from datetime import datetime as dt
def to_timestamp(timestr):
    dtobj = dt.strptime(timestr, '%Y-%m-%d %H:%M:%S')
    timestamp = int(dtobj.timestamp())
    return timestamp

'''
`id` bigint(20) NOT NULL,
`site_id` bigint(20) NOT NULL,
`link_name` varchar(255) CHARACTER SET utf8 NOT NULL,
`pic_url` varchar(255) CHARACTER SET utf8 DEFAULT NULL,
`date_created` datetime NOT NULL,
`genres_names_string` varchar(300) CHARACTER SET utf8 DEFAULT NULL,
`tags_names_string` var
'''

def parse_row(row):
    quote_opened = False
    content = []
    curr_content = ''
    for symb in row:
        if quote_opened:
            if symb != "'":
                curr_content += symb
            else:
                quote_opened = False
                content.append(curr_content)
                curr_content = ''

        else:
            if symb == "'":
                quote_opened = True
            elif symb == ',':
                # comma-separated value. Not comma inside quote
                if len(curr_content) > 0:
                    content.append(curr_content)
                    curr_content = ''
            else:
                curr_content += symb
    if len(curr_content):
        content.append(curr_content)
        
    return content

def cast_row_objects(row):
    for i, item in enumerate(row):
        if all(map(str.isdigit, item)):
            row[i] = int(item)
        elif item == 'NULL':
            row[i] = None
        elif ', ' in item:
            row[i] = item.split(', ')
    return row


def read_dump(dump_filename, target_table):
    fast_forward = True
    buffer = []
    
    with open(dump_filename, 'r') as fin:
#         with open(csv_path, 'w') as fout:

        for line in tqdm(fin):
            line = line.strip()
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
            if fast_forward:
                continue

            data = re.findall('\([^\)]*\)', line)
            for i, obj in enumerate(data):
                # (254995,6,'strashno_prekrasen','https://static...','2020-10-...','fantasy, love','bulvarnaia_proza, proza')
                try:
                    row = obj[1:-1] # drop brackets
                    row = parse_row(row)
                    if len(row) != 7:
                        continue
                    
                    useful_cols_ixs = (0, 1, 5, 6)                        
                    row = [row[ix] for ix in useful_cols_ixs]
                    row = cast_row_objects(row)
                    buffer.append(row)
#                         if len(buffer) > BUFFER_SIZE:
#                             fout.write('\n'.join(buffer) + '\n') # write out
#                             buffer = []

                except Exception as e:
                    pass#raise e
                    
#             fout.write('\n'.join(buffer)) # last data writing
    return buffer

In [3]:
path = '/data/groupLe_recsys/raw/all-genres.sql'

data = read_dump(path, 'el_recomm')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


In [4]:
print(len(data))
print(data[0])

143224
[2581, 3, ['comedy', 'shounen', 'supernatural'], None]


In [5]:
site_ids = {'dorama': 5,
            'manga':  1,
            'mint': 2,
            'book':   6}

path_pattern = '../../data/interim/{}/genres.json'

for site_name, site_id in site_ids.items():
    jsons = []
    
    for row in data:
        if row[1] != site_id:
            continue
            
        def validate(collection):
            # collection may be list of items or item or None
            if collection is None:
                return []
            if isinstance(collection, str):
                return [collection]
            else:
                return collection
            
        genres = validate(row[2])
        tags = validate(row[3])
        marks = list(set(genres + tags))
        marks = genres
        obj = {'id': row[0],
               'genres': marks}
        jsons.append(obj)
        
    path = path_pattern.format(site_name)
    json.dump(jsons, open(path, 'w'))
    
    print('written %d object for %s' % (len(jsons), site_name))

written 20813 object for dorama
written 20087 object for manga
written 13110 object for mint
written 86710 object for book


In [6]:
books = list(filter(lambda row: row[1] == 6, data))
books[:30]

[[234705, 6, 'detective', 'classic'],
 [111845, 6, ['animal', 'adventure'], None],
 [108069, 6, ['animal', 'adventure'], None],
 [81132, 6, ['adventure', 'animal'], None],
 [62435, 6, ['adventure', 'animal'], 'children'],
 [21593,
  6,
  ['animal', 'psychology', 'adventure'],
  ['children', 'rus_classic']],
 [224275, 6, 'drama', 'children'],
 [211605, 6, 'military', 'children'],
 [177119, 6, None, 'children'],
 [158296, 6, ['animal', 'adventure'], 'children'],
 [123715, 6, None, 'children'],
 [95892, 6, ['animal', 'adventure'], None],
 [35407, 6, None, 'children'],
 [200193, 6, None, 'children'],
 [233314,
  6,
  ['humor', 'antique_myths', 'horror'],
  ['children', 'rus_classic']],
 [194587, 6, None, 'children'],
 [227199, 6, None, 'children'],
 [41666, 6, None, 'children'],
 [212129, 6, None, 'children'],
 [205611, 6, 'sf', 'children'],
 [151102, 6, None, None],
 [30025, 6, 'sf', ['children', 'sovremennaia_russkaia_proza']],
 [132350, 6, ['travel', 'adventure'], None],
 [151734, 6, 's

In [7]:
gs = set()
for item in jsons:
    for g in item['genres']:
        gs.add(g)

In [8]:
len(gs)

84

In [9]:
gs

{'absurd',
 'action',
 'adventure',
 'alternativnaia_istoriia',
 'animal',
 'antique_myths',
 'art',
 'astronomiia',
 'autobiographic',
 'biographic',
 'counter',
 'crime',
 'cyberpunk',
 'detective',
 'documentary',
 'drama',
 'dystopia',
 'ekonomika',
 'epic',
 'erotika',
 'espionage',
 'fantasy',
 'fizika',
 'gorodskoe_fentezi',
 'gotika',
 'hard',
 'heroic',
 'history',
 'horror',
 'hronoopera',
 'humor',
 'irony',
 'kitchen',
 'kosmicheskaia_fantastika',
 'kosmicheskaia_opera',
 'kulturologiia',
 'literaturovedenie',
 'love',
 'love_relation',
 'magicheskii_realizm',
 'maniac',
 'medicina',
 'melodrama',
 'military',
 'morskie_prikliucheniia',
 'mystery',
 'mystik',
 'nauchnaia_fantastika',
 'noir',
 'parodiia',
 'philosophy',
 'police',
 'political',
 'popular_science',
 'postapocalyptic',
 'postmodernizm',
 'povsednevnost',
 'psihodelika',
 'psihologiia',
 'psychology',
 'realizm',
 'religion',
 'retellings',
 'roman_vospitaniia',
 'romantika',
 'samosovershenstvovanie',
 'saspe